# Exam Planning-Lab

## Exercise 1

Consider the following environment:

<img src="images/env1.png" alt="ex1" style="zoom: 40%;" />

The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls of the previous environments, the floor is covered with lava, however the agent can resist to high temperature and it can use the heat to recharge its batteries, hence it receives a positive reward for being on a lava cell. The environment is deterministic and the agent must avoid the black pits of death (cells $(0,3)$, $(1, 3)$, $(1,1)$). 
Assume that you do not have access to the motion model and to reward and that the problem is undiscounted (i.e., $\gamma$=1)


In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
from timeit import default_timer as timer
from tqdm import tqdm as tqdm
import numpy as np

def epsilon_greedy(q, state, epsilon):
    """
    Epsilon-greedy action selection function
    
    Args:
        q: q table
        state: agent's current state
        epsilon: epsilon parameter
    
    Returns:
        action id
    """
    an = q.shape[1]
    probs = np.full(an, epsilon / an)
    probs[q[state].argmax()] += 1 - epsilon
    return np.random.choice(an, p=probs)


env = gym.make('LavaFloor-v0')
env.render()

[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]


### 1.1) Given the environment reported above, find a policy with a suitable algorithm of your choice. Print the resulting policy (you can use the provided code for this)


In [15]:
def q_learning(environment, episodes, alpha, gamma, expl_func, expl_param):
    """
    Performs the Q-Learning algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        episodes: number of episodes for training
        alpha: alpha parameter
        gamma: gamma parameter
        expl_func: exploration function (epsilon_greedy, softmax)
        expl_param: exploration parameter (epsilon, T)
    
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """
    
    q = np.zeros((environment.observation_space.n, environment.action_space.n))  # Q(s, a)
    rews = np.zeros(episodes)
    lengths = np.zeros(episodes)
    env = environment
    for e in range(episodes):
        state = env.reset()
        done = False
        leng = 0
        while not done:
           #choose  action
            action = expl_func(q, state, expl_param)
            
            new_state, reward, done, _ = env.step(action)
            rews[e] += reward
            
            max_ = q[new_state,action]
            for a in range(environment.action_space.n):
                if q[new_state,a] > max_:
                    max_ = q[new_state,a]
                
            q[state,action] = q[state,action] + alpha * (reward + gamma *max_  - q[state,action])
            state = new_state
            leng+=1
        
        lengths[e] = leng
            
    policy = q.argmax(axis=1)
    return policy, rews, lengths

def sarsa(environment, episodes, alpha, gamma, expl_func, expl_param):
    """
    Performs the SARSA algorithm for a specific environment
    
    Args:
        environment: OpenAI gym environment
        episodes: number of episodes for training
        alpha: alpha parameter
        gamma: gamma parameter
        expl_func: exploration function (epsilon_greedy, softmax)
        expl_param: exploration parameter (epsilon, T)
    
    Returns:
        (policy, rewards, lengths): final policy, rewards for each episode [array], length of each episode [array]
    """

    q = np.zeros((environment.observation_space.n, environment.action_space.n))  # Q(s, a)
    rews = np.zeros(episodes)
    lengths = np.zeros(episodes)
    for i in range(episodes):
        state = environment.reset()  # Reset the environment
        action = expl_func(q, state, expl_param)  # Select first action
        el = 0
        
        while True:
            next_state, reward, done, _ = environment.step(action)  # Execute a step
            action_next = expl_func(q, next_state, expl_param)  # Select next action
            q[state, action] += alpha * (reward + gamma * q[next_state, action_next] - q[state, action])  # Temporal difference
            rews[i] += reward
            el += 1
            
            if done:
                break
                
            state = next_state  # Update state
            action = action_next  # Update action
            
        lengths[i] = el
        
    policy = q.argmax(axis=1) # q.argmax(axis=1) automatically extract the policy from the q table
    return policy, rews, lengths


In [18]:
episodes = 10
gamma = .99
alpha = .3
epsilon = .1


env = gym.make('LavaFloor-v0')
policy, rewards, lengths = q_learning(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Policy:\n {} \n".format(np.vectorize(env.actions.get)(policy.reshape(env.shape))))

Policy:
 [['R' 'D' 'U' 'U']
 ['D' 'L' 'U' 'L']
 ['R' 'R' 'R' 'L']] 



### 1.2) Find a way to force the agent to choose the shortest path towards the goal. You can change these parameters: episodes, alpha, gamma exploration function and exploration parameter. 

##### Hint: you should tune a parameter to consider the immediate reward rather than the long-term one... 

In [19]:
episodes = 300 # your code here
gamma =  .99 #your code here # lower gamma will make the robot focus on short term rews
alpha = .3
epsilon = .1


# Q-Learning or SARSA epsilon greedy
env = gym.make('LavaFloor-v0')
p, r, l = sarsa(env, episodes, alpha, gamma, epsilon_greedy, epsilon)
print("Policy:\n {} \n".format(np.vectorize(env.actions.get)(p.reshape(env.shape))))

Policy:
 [['D' 'L' 'L' 'R']
 ['D' 'L' 'U' 'L']
 ['R' 'R' 'R' 'L']] 



# 1.3) Consider and environment with states {A, B, C, D}, actions {r, l} where states {A, D} are terminal. Consider the following sequence of learning episodes:
* E1: (B, r, C, −1),(C, r, C, −1),(C, r, D, +1)
* E2: (B, r, C, −1),(C, r, D, +1)
* E3: (B, l, A, +5)
* E4: (B, l, B, −1),(B, l, B, −1),(B, l, A, +5)
### Compute v(s) for all non-terminal states by using a sample-based evaluation approach (i.e., computing the values with the function reported below). Assume $\alpha$ = .5 and $\gamma$ = 1.
### $V(s) = (1- \alpha) \cdot V(s) + \alpha \cdot (r + \gamma \cdot V'(s'))$ 
### where $s$ is the state under consideration (first element of each tuple), $s'$  (third element of each tuple ) the state reached by starting from $s$ and performing the action $a$ (second element of each tuple). And finally, $r$ is the reward (last element of each tuple).

In [ ]:
episodes = {1: [('B', 'r', 'C', -1), ('C', 'r', 'C', -1),('C', 'r', 'D', 1)], 
            2: [('B', 'r', 'C', -1), ('C', 'r', 'D', 1)], 
            3: [('B', 'l', 'A', 5)], 
            4: [('B', 'l', 'B', -1),('B', 'l', 'B', -1),('B', 'l', 'A', 5)]}

v = {'A': 5, 'B': 0, 'C': 0, 'D': 1}
alpha = 0.5
gamma = 1


for episode, values in episodes.items():
    for tuple in values:
        # your code here
        ...

print(v)

## Exercise 2

Consider the figure below where **S=$(1,2)$** and **G=$(3,1)$** are the starting and goal positions respectively. Consider the problem of finding a minimum cost path from S to G assuming the agent can move in the four directions (if there is no
obstacle) and that each movement has a unitary cost. The environment is deterministic. Answer the following questions:

<img src="images/ex2.png" alt="ex2" style="zoom: 40%;" />


In [ ]:
env = gym.make('SmallMaze-v0')
env.render()

### 2.1) Verify by using the code developed in the lab that the Manhattan distance (l1_norm) is a  *consistent* heuristic in this environment. In particular, you should implement a function that checks whether for every couple of state $(s,s')$ (where $s'$ is a successor state of $s$), the consistency condition is verified. The function should return true if the heuristic is consistent and false otherwise. Recall that every action has a cost of 1...


In [ ]:
# your code here

### 2.2) Consider the A* algorithm and assume want to achieve optimality. Based on the *consistent* heuristics of Section 2.1, state whether it is best to use a graph search or tree search strategy. Motivate your answer and show the results of A* execution and the differences between the two versions in terms of the computed solution, number of nodes generated and maximum number of nodes in memory. 


Since we have a consistent heuristic we have to choose a graph search version to achieve optimality. We know only if tree search + admissible heuristic or graph search + consistent heuristic => optimality of A*. In all other situations, we cannot guarantee optimality.

In [ ]:
def present_with_higher_cost(queue, node):
    if node.state in queue:
        if queue[node.state].value > node.value: 
            return True
    return False

In [ ]:
# your code here

In [ ]:
env = gym.make('SmallMaze-v0')
solution_ts, time_ts, memory_ts = astar_tree_search(env)
solution_gs, time_gs, memory_gs = astar_graph_search(env)

print("Solution (tree-search): {}".format(solution_2_string(solution_ts, env)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}\n".format(memory_ts))
print('='*65)
print("\nSolution (graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

### 2.3) Let us consider BFS, show the path of the optimal solution (avoiding the repetition of states) to achieve the goal. In this scenario can we guarantee that the returned solution is the least cost one? If we used Iterative Deepening Search(IDS) with the same methodology used for BFS can we guarantee a least cost solution? Justify your answer and show the results of the different strategies by using the code developed during the lab. 


In [ ]:
# your BFS code here

In [ ]:
env = gym.make('SmallMaze-v0')
solution_gs, time_gs, memory_gs = BFS_GraphSearch(env)
print("Solution (BFS graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

In [ ]:
#example IDS+GRAPH SEARCH


In [ ]:
env = gym.make('SmallMaze-v0')
solution_gs, time_gs, memory_gs, iterations_gs = IDS(env, Recursive_DLS_GraphSearch)
print("Solution (IDS graph-search): {}".format(solution_2_string(solution_gs, env)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}\n".format(memory_gs))

### Discuss the results achieved with respect to the 2.3 question:

## Exercise 3

Consider the environment displayed in Figure below, where states $(0, 3)$ and $(1, 3)$ are terminal states with reward $+1$ and $−1$ respectively. The agent can move in the four directions. The transition model states that for every state and action the agent has $0.8$ chances of moving in the chosen direction and $0.1$ chances to move in the othogonal directions.The reward model states that for every state, action and successor state the agent pays $−0.04$. Assume that the dicount factor is set to $0.9$. Answer the following questions: 

<img src="images/ex3.1.png" alt="ex3" style="zoom: 40%;" />


In [ ]:
env = gym.make('VeryBadLavaFloor-v0')
env.render()

### 3.1) Use one of the methods developed in the lab to compute the value function for the left diagram. 

In [ ]:
# your code here

In [ ]:
env = gym.make('VeryBadLavaFloor-v0')
policy, values = ... # your code here
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(values)
print(policy_render)


### Results:

<img src="images/ex3.png" alt="ex3" style="zoom: 40%;" />

### 3.2) Consider the right diagram and focus on states $(2, 1)$. State whether the action reported in the right diagram (the blue one) represents the optimal action for that state. Motivate your answer with the code...

In [ ]:
actions = {0: "L", 1: "R", 2: "U", 3: "D"}

value_function = [0.50939438, 0.64958568, 0.79536209, 1, 0.39844322, 0, 0.48644002, -1, 0.29628832,  0.253867, 0.34475423, 0.12987275]
id_start_state = 9
gamma = 0.9


values_ex = [0, 0, 0, 0]
'''
YOUR CODE HERE
'''
    
print(np.asarray(values_ex))
print(f'The correct action to perform should be: {actions[np.argmax(values_ex)]}')

### 3.3) Compute the probability of ending in state (1, 3) if we execute the sequence of actons < Up, Up > from state (2, 2). Motivate your answer reporting the code and the solution printed. The following image shows a diagram to guide the solution process as a hint for you:


<img src="images/example.png" alt="ex3" style="zoom: 30%;" />



In [ ]:
id_start_state = 10
state = id_start_state
actions = {0: "L", 1: "R", 2: "U", 3: "D"}

prob = 0
action = 2
probs_fin = 0

for next_state in range(env.observation_space.n):
    
    if env.T[state, action, next_state] == 0:
        continue


    # your code here
    ...

    for second_next_state in range(env.observation_space.n):
        
        if env.T[next_state, action, second_next_state] == 0:
            continue

        if env.grid[second_next_state] == "P": #check that the state is (1,3) i.e., the black pit
            # your code here
            ...

            print(f'{env.state_to_pos(state)} --> {env.state_to_pos(next_state)} --> {env.state_to_pos(second_next_state)}')
            print(f'probs: {env.T[state, action, next_state]} --> {env.T[next_state, action, second_next_state]}')

            # your code here
            ...

            print('================')
            break
    
print()
print('Probability: ', probs_fin)  
    



### 3.4) Consider the following environment where states $(0, 3)$ and $(1, 3)$ are terminal states with reward $+1$ and $−1$ respectively. Assume the transition model is the same one defined above and that the discounted factor is $0.9$ as above. However, now the agent pays $−0.4$ instead of $−0.04$ for every action, state and successor state.  Compute the new value function and the optimal policy for this new environment. 


<img src="images/env_3.4.png" alt="ex3" style="zoom: 40%;" />

In [ ]:
env = gym.make('NiceLavaFloor-v0')
policy, values = ... # yout code here
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(values)
print(policy_render)

### 3.5) Modify the value iteration parameters so that the policy allows the agent to reach the goal only starting from states $(0,1)$, $(0,2)$ and $(1,2)$, as reported in the image below . Motivate your answer.


#### hint: given the negative reward the agent should care about the immediate reward and not the long-term reward.



<img src="images/result_ex3.5.png" alt="ex3" style="zoom: 40%;" />

In [ ]:
env = gym.make('NiceLavaFloor-v0')
policy, values =  ... #your code here
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
print(policy_render)